In [5]:
import rpy2
from rpy2.robjects.packages import importr

print(rpy2.__version__)

# import R's "base" package
base = importr('base')

# import R's "utils" package
utils = importr('utils')

datavolley = importr('datavolley')
ovlytics = importr('ovlytics')
aws_s3 = importr('aws.s3')
dplyr = importr('dplyr')
jsonlite = importr('jsonlite')
purrr = importr('purrr')


px = rpy2.robjects.r('''
bucket_df <- get_bucket_df('datavolley',prefix='rseq/',region="",use_https=FALSE)
lapply(bucket_df[["Key"]], function(x) {
  filename <- paste("dvw/",x, sep="")
  save_object(
    x,
    'datavolley',
    file = filename,
    overwrite = TRUE,
    region="",
    use_https=FALSE
  )
})

# directory containing all list of dvw files
d <- dir("./dvw/rseq", pattern = "dvw$", full.names = TRUE)
# read each file
lx <- lapply(d, dv_read, insert_technical_timeouts = FALSE)

# open duckDB connection
# con <- duckdb::dbConnect(duckdb::duckdb(),"out/datavolley.db")

# remap inconsitent team names to team_id
lx <- remap_team_names(lx, data.frame(from="DALHOUSIE TIGERS", to="M-DALHOUSIE TIGERS", team_id="MRDT"))
lx <- remap_team_names(lx, data.frame(from="MONTREAL", to="M-MONTREAL CARABINS", team_id="MRMC"))
lx <- remap_team_names(lx, data.frame(from="UNB", to="M-UNB REDS", team_id="MRUR"))


# combined plays object
px <- bind_rows(lapply(lx, plays))

# write.csv(px,"plays.csv", row.names = FALSE)
# put_object(file="plays.csv",object="rseq/plays.csv",bucket='datavolley',region="",use_https=FALSE)

json_data <- toJSON(lx, pretty=TRUE)

# Initialize empty dataframes to store aggregated results
players_df <- data.frame() 

# Apply function to each element in lx

# Iteratively combine each element
for (x in lx) {
  players <- bind_rows(x[["meta"]][["players_v"]], x[["meta"]][["players_h"]])
  players_df <- Reduce(function(x, y) bind_rows(x, y) %>% distinct(player_id, number, firstname,lastname,role), list(players,players_df))
}

# augment plays using ovlytics
augmented_px <- ov_augment_plays(
  px,
  to_add = c("touch_summaries", "setters"),
  use_existing = TRUE
)
''')

3.5.11


In [9]:
from rpy2.robjects import pandas2ri
import rpy2.robjects as ro

with (ro.default_converter + pandas2ri.converter).context():
  pd_from_plays = ro.conversion.get_conversion().rpy2py(ro.r.augmented_px)

pd_from_plays.head(10)

,match_id,point_id,time,video_file_number,video_time,code,team,player_number,player_name,player_id,...,visiting_score_start_of_point,freeball_over,ts_pass_quality,ts_pass_evaluation_code,ts_set_error,home_setter_id,visiting_setter_id,setter_id,setter_position,setter_front_back
1,eb01535dcbdcab63a67fa299ec438793,0.0,NaT,1.0,NaN,*P01>LUp,M-MONTREAL CARABINS,-2147483648,NA_character_,NA_character_,...,0,False,NA_character_,NA_character_,False,NA_character_,NA_character_,NA_character_,-2147483648,NA_character_
2,eb01535dcbdcab63a67fa299ec438793,0.0,NaT,1.0,NaN,*z5>LUp,M-MONTREAL CARABINS,-2147483648,NA_character_,NA_character_,...,0,False,NA_character_,NA_character_,False,STD-MAX,NA_character_,STD-MAX,5,back
3,eb01535dcbdcab63a67fa299ec438793,0.0,NaT,1.0,NaN,aP07>LUp,M-UNB REDS,-2147483648,NA_character_,NA_character_,...,0,False,NA_character_,NA_character_,False,STD-MAX,NA_character_,NA_character_,-2147483648,NA_character_
4,eb01535dcbdcab63a67fa299ec438793,0.0,NaT,1.0,NaN,az2>LUp,M-UNB REDS,-2147483648,NA_character_,NA_character_,...,0,False,NA_character_,NA_character_,False,STD-MAX,MAF-LUC,MAF-LUC,2,front
5,eb01535dcbdcab63a67fa299ec438793,0.0,2025-01-11 08:34:45+00:00,1.0,1343.0,*06SM+~~~66,M-MONTREAL CARABINS,6,GHASSEN ZGARNI,ZGA-GHA,...,0,False,NA_character_,NA_character_,False,STD-MAX,MAF-LUC,STD-MAX,5,back
6,eb01535dcbdcab63a67fa299ec438793,0.0,2025-01-11 08:34:55+00:00,1.0,1344.0,a04RM-~~~66,M-UNB REDS,4,EDUARDO LUCHESE,LUC-EDU,...,0,False,Poor,-,False,STD-MAX,MAF-LUC,MAF-LUC,2,front
7,eb01535dcbdcab63a67fa299ec438793,0.0,2025-01-11 08:34:58+00:00,1.0,1346.0,a05EH#~~F,M-UNB REDS,5,SCOTT MANN,MAN-SCO,...,0,False,Poor,-,False,STD-MAX,MAF-LUC,MAF-LUC,2,front
8,eb01535dcbdcab63a67fa299ec438793,0.0,2025-01-11 08:35:02+00:00,1.0,1348.0,a04AH#V5~45~H3,M-UNB REDS,4,EDUARDO LUCHESE,LUC-EDU,...,0,False,Poor,-,False,STD-MAX,MAF-LUC,MAF-LUC,2,front
9,eb01535dcbdcab63a67fa299ec438793,0.0,2025-01-11 08:35:02+00:00,1.0,1348.0,*$$&H=,M-MONTREAL CARABINS,-2147483648,NA_character_,NA_character_,...,0,False,NA_character_,NA_character_,False,STD-MAX,MAF-LUC,STD-MAX,5,back
10,eb01535dcbdcab63a67fa299ec438793,0.0,2025-01-11 08:35:12+00:00,1.0,1358.0,ap00:01,M-UNB REDS,-2147483648,NA_character_,NA_character_,...,0,False,NA_character_,NA_character_,False,STD-MAX,MAF-LUC,MAF-LUC,2,front
